###### 由于favorite是目前比较有用的feature。这里需要总结几个目前想到的特征: 
1. 根据entiti去总结出被引用最多的50种entitie，然后算他们出现时的retweet
2. 根据hashtage总结出引用最多的50种hashtag，然后他们出现时的retweet
3. 根据favorite筛选出其中的最大值以及最小值
4. 根据favorite选出最常搭配的10个entitie，10个hashtag

In [ ]:
#invite people for the Kaggle party
import os
import gc
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
ROOT_PATH = '/home/chai/Projects/CIKM2020'
INPUT_PATH = os.path.join(ROOT_PATH, 'public_dat_50027')

In [ ]:
# read the train data and train solution
import time
X_train_list = []
with open(os.path.join(INPUT_PATH, 'train.data'), 'r') as f:
    for line in f.readlines():
        X_train_list.append(line.strip('\n').split('\t'))

In [ ]:
#read the solution
y_train_list = []
with open(os.path.join(INPUT_PATH, 'train.solution'), 'r') as f:
    for line in f.readlines():
        y_train_list.append([line.strip('\n')])

In [ ]:
#get the column
feature_name = pd.read_table(os.path.join(INPUT_PATH, 'feature.name'), sep='\t')
train_list = []
for i, item in enumerate(X_train_list):
    if np.random.random() < 0.01:
        train_list.append(item + [int(y_train_list[i][0])])
#     if i > 50000:
#         break
#     if item[5] != '0':
#         item.append(solution[i][0])
#         train.append(item)

In [ ]:
print('采样率为: ', len(train_list) / len(X_train_list))

In [ ]:
del X_train_list, y_train_list
gc.collect()

In [ ]:
train_data = pd.DataFrame(train_list, columns=feature_name.columns.tolist() + ['solution'])

train_data['weekday'] = train_data['timestamp'].map(lambda x: x.split(" ")[0])
train_data['month'] = train_data['timestamp'].map(lambda x: x.split(" ")[1])
train_data['day'] = train_data['timestamp'].map(lambda x: int(x.split(" ")[2]))
train_data['hour'] = train_data['timestamp'].map(lambda x: int(x.split(" ")[3].split(":")[0]))
train_data['year'] = train_data['timestamp'].map(lambda x: int(x.split(" ")[-1]))
week_day = {"Mon": 1,"Tue":2,"Wed":3,"Thu":4,"Fri":5,"Sat":6,"Sun":7}
month_day = {"Sep":9,"Oct":10,"Nov":11,"Dec":12,"Jan":1,"Feb":2,"Mar":3,"Apr":4}
train_data['weekday'] = train_data['weekday'].replace(week_day)
train_data['month'] = train_data['month'].replace(month_day)
# train_data = train_data.drop(columns=['timestamp'])

train_data['#followers'] = train_data['#followers'].astype(int)
train_data['#friends'] = train_data['#friends'].astype(int)
train_data['#favorites'] = train_data['#favorites'].astype(int)

In [ ]:
# favorites为0，但是solution不为0的个数
df0 = train_data.loc[train_data['#favorites'] == 0]
df1 = df0.loc[df0['solution'] > 0]
print('favorites为0，但是solution不为0的个数: ', df1.shape[0], ', 比例为: ', df1.shape[0] / df0.shape[0])
df0[['#followers', '#friends', '#favorites', 'solution']].describe()

In [ ]:
print('Total: ', len(train_data))
train_data.nunique()

In [ ]:
df = 10000 * train_data['year'] + 100 * train_data['month'] + train_data['day']
print('最早的时间:', df.min(), ', 最晚的时间:', df.max())

In [ ]:
start_day = datetime.strptime('2019-09-30', '%Y-%m-%d')
train_data['timestamp'] = train_data['timestamp'].map(lambda x : (datetime.strptime(x, '%a %b %d %H:%M:%S +0000 %Y') - start_day).days)

In [ ]:
train_data.info()

In [ ]:
train_data['sentiment_left'] = train_data['sentiment'].map(lambda x : int(x.split(' ')[0]))
train_data['sentiment_right'] = train_data['sentiment'].map(lambda x : int(x.split(' ')[1]))
train_data['sentiment_sum'] = train_data['sentiment_left'] + train_data['sentiment_right']
train_data['sentiment_sumabs'] = (train_data['sentiment_left'] + train_data['sentiment_right']).map(lambda x : abs(x))
train_data['sentiment_diff'] = train_data['sentiment_left'] - train_data['sentiment_right']
train_data['sentiment_div'] = train_data['sentiment_left'] / train_data['sentiment_right']

In [ ]:
for var in ['sentiment_left', 'sentiment_right', 'sentiment_sum', 'sentiment_sumabs', 'sentiment_diff', 'sentiment_div']:
    print(var)
    print(train_data[var].value_counts())
    data = pd.concat([train_data['solution'], train_data[var]], axis=1)
    f, ax = plt.subplots(figsize=(8, 6))
    fig = sns.boxplot(x=var, y="solution", data=data)
    fig.axis(ymin=0, ymax=40);
    plt.show()

In [ ]:
#saleprice correlation matrix
corrmat = train_data[['solution', 'sentiment_left', 'sentiment_right', 'sentiment_sum', 'sentiment_sumabs', 'sentiment_diff', 'sentiment_div']].corr()
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'solution')['solution'].index
cm = np.corrcoef(train_data[cols].values.T)
f, ax = plt.subplots(figsize=(6, 6))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
train_data.loc[train_data['mentions'] != 'null;']['mentions'].head(30)

In [ ]:
print('特征mentions不为null;的值有:', train_data.loc[train_data['mentions'] != 'null;'].shape[0])
print('比例为:', train_data.loc[train_data['mentions'] != 'null;'].shape[0] / train_data.shape[0])
print('特征mentions的数量为:', train_data['mentions'].nunique())

In [ ]:
train_data.loc[train_data['entities'] != 'null;']['entities'].head(10)

In [ ]:
entities_list = []
for each_entity in train_data['entities']:
    if each_entity != 'null;':
        for each in each_entity.strip(';').split(';'):
            entities_list.append(float(each.split(':')[-1]))

In [ ]:
print('特征entities不为null;的值有:', train_data.loc[train_data['entities'] != 'null;'].shape[0])
print('比例为:', train_data.loc[train_data['entities'] != 'null;'].shape[0] / train_data.shape[0])
print('特征entities的数量为:', train_data['entities'].nunique())

In [ ]:
print('特征entities的最大值:', max(entities_list))
print('特征entities的最小值:', min(entities_list))
print('特征entities的均值:', np.mean(entities_list))

In [ ]:
entities_mean = []
for each_entity in train_data['entities']:
    if each_entity != 'null;':
        tmp = []
        for each in each_entity.strip(';').split(';'):
            tmp.append(float(each.split(':')[-1]))
        entities_mean.append(np.mean(tmp))
    else:
        entities_mean.append(0)
train_data['entities_mean'] = entities_mean

In [ ]:
train_data['entities_mean']

In [ ]:
entities_max = []
for each_entity in train_data['entities']:
    if each_entity != 'null;':
        tmp = []
        for each in each_entity.strip(';').split(';'):
            tmp.append(float(each.split(':')[-1]))
        entities_max.append(max(tmp))
    else:
        entities_max.append(0)
train_data['entities_max'] = entities_max

In [ ]:
entities_min = []
for each_entity in train_data['entities']:
    if each_entity != 'null;':
        tmp = []
        for each in each_entity.strip(';').split(';'):
            tmp.append(float(each.split(':')[-1]))
        entities_min.append(max(tmp))
    else:
        entities_min.append(0)
train_data['entities_min'] = entities_min

In [ ]:
entities_std = []
for each_entity in train_data['entities']:
    if each_entity != 'null;':
        tmp = []
        for each in each_entity.strip(';').split(';'):
            tmp.append(float(each.split(':')[-1]))
        entities_std.append(np.std(tmp))
    else:
        entities_std.append(0)
train_data['entities_std'] = entities_std

In [ ]:
entities_len = []
for each_entity in train_data['entities']:
    if each_entity != 'null;':
        entities_len.append(len(each_entity.strip(';').split(';')))
    else:
        entities_len.append(0)
train_data['entities_len'] = entities_len

In [ ]:
#saleprice correlation matrix
corrmat = train_data[['solution', 'entities_mean', 'entities_max', 'entities_min', 'entities_std', 'entities_len']].corr()
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'solution')['solution'].index
cm = np.corrcoef(train_data[cols].values.T)
f, ax = plt.subplots(figsize=(6, 6))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
print('特征mentions不为null;的值有:', train_data.loc[train_data['mentions'] != 'null;'].shape[0])
print('比例为:', train_data.loc[train_data['mentions'] != 'null;'].shape[0] / train_data.shape[0])
print('特征mentions的数量为:', train_data['mentions'].nunique())

In [ ]:
train_data['mentions'].head(30)

In [ ]:
mentions_count_dict = {}
for each_mention in train_data['mentions']:
    if each_mention != 'null;':
        for each in each_mention.strip().split(' '):
            mentions_count_dict[each] = mentions_count_dict.get(each, 0) + 1

In [ ]:
mentions_len = []
for each_mention in train_data['mentions']:
    if each_mention != 'null;':
        mentions_len.append(len(each_mention.split(' ')))
    else:
        mentions_len.append(0)
train_data['mentions_len'] = mentions_len

In [ ]:
mentions_value= []
for each_mention in train_data['mentions']:
    if each_mention != 'null;':
        tmp = 0
        for each in each_mention.strip().split(' '):
            tmp += mentions_count_dict[each]
        mentions_value.append(tmp)
    else:
        mentions_value.append(0)
train_data['mentions_value'] = mentions_value

In [ ]:
#saleprice correlation matrix
corrmat = train_data[['solution', 'mentions_len', 'mentions_value']].corr()
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'solution')['solution'].index
cm = np.corrcoef(train_data[cols].values.T)
f, ax = plt.subplots(figsize=(6, 6))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
print('特征hashtags不为null;的值有:', train_data.loc[train_data['hashtags'] != 'null;'].shape[0])
print('比例为:', train_data.loc[train_data['hashtags'] != 'null;'].shape[0] / train_data.shape[0])
print('特征hashtags的数量为:', train_data['hashtags'].nunique())

In [ ]:
hashtags_count_dict = {}
for each_hashtag in train_data['hashtags']:
    if each_hashtag != 'null;':
        for each in each_hashtag.strip().split(' '):
            hashtags_count_dict[each] = hashtags_count_dict.get(each, 0) + 1

In [ ]:
hashtags_len = []
for each_hashtag in train_data['hashtags']:
    if each_hashtag != 'null;':
        hashtags_len.append(len(each_hashtag.split(' ')))
    else:
        hashtags_len.append(0)
train_data['hashtags_len'] = hashtags_len

In [ ]:
hashtags_value= []
for each_hashtag in train_data['hashtags']:
    if each_hashtag != 'null;':
        tmp = 0
        for each in each_hashtag.strip().split(' '):
            tmp += hashtags_count_dict[each]
        hashtags_value.append(tmp)
    else:
        hashtags_value.append(0)
train_data['hashtags_value'] = hashtags_value

In [ ]:
#saleprice correlation matrix
corrmat = train_data[['solution', 'hashtags_len', 'hashtags_value']].corr()
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'solution')['solution'].index
cm = np.corrcoef(train_data[cols].values.T)
f, ax = plt.subplots(figsize=(6, 6))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
train_data['urls_count'] = train_data['urls'].map(lambda x : 1 if x != 'null;' else 0)

In [ ]:
# class_mapping = {label:idx for idx,label in enumerate(set(train_data['entities']))}
# train_data['entities'] = train_data['entities'].map(class_mapping)
# class_mapping = {label:idx for idx,label in enumerate(set(train_data['sentiment']))}
# train_data['sentiment'] = train_data['sentiment'].map(class_mapping)
# class_mapping = {label:idx for idx,label in enumerate(set(train_data['mentions']))}
# train_data['mentions'] = train_data['mentions'].map(class_mapping)
# class_mapping = {label:idx for idx,label in enumerate(set(train_data['hashtags']))}
# train_data['hashtags'] = train_data['hashtags'].map(class_mapping)
# class_mapping = {label:idx for idx,label in enumerate(set(train_data['urls']))}
# train_data['urls'] = train_data['urls'].map(class_mapping)

In [ ]:
train_data[['solution','#followers','#friends','#favorites']].describe()

In [ ]:
print("Skewness: %f" % train_data['solution'].skew())
print("Kurtosis: %f" % train_data['solution'].kurt())

In [ ]:
sns.distplot(train_data['solution']);

In [ ]:
sns.distplot(train_data.loc[train_data['solution'] < 30]['solution']);

In [ ]:
sns.distplot(train_data[train_data['#favorites'] == 30]['solution'])

难以根据日期来判断solution的结果，并且solution存在大量的异常点

In [ ]:
var = 'timestamp'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
data.plot.scatter(x=var, y='solution', ylim=(0,20));

In [ ]:
var = 'weekday'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
#data.plot.scatter(x=var, y='solution', ylim=(0,300000));
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="solution", data=data)
fig.axis(ymin=0, ymax=50);

看上去9月的solution更集中，但是9月的转发远少于10月

In [ ]:
var = 'month'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
#data.plot.scatter(x=var, y='solution', ylim=(0,300000));
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="solution", data=data)
fig.axis(ymin=0, ymax=30);
print(train_data[var].value_counts())

In [ ]:
var = 'day'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
#data.plot.scatter(x=var, y='solution', ylim=(0,300000));
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="solution", data=data)
fig.axis(ymin=0, ymax=30);

In [ ]:
var = 'hour'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
#data.plot.scatter(x=var, y='solution', ylim=(0,300000));
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="solution", data=data)
fig.axis(ymin=0, ymax=30);

sentiment不该如此简单的处理

In [ ]:
var = 'sentiment'
print(train_data[var].value_counts())
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
#data.plot.scatter(x=var, y='solution', ylim=(0,300000));
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="solution", data=data)
fig.axis(ymin=0, ymax=100);

In [ ]:
var = 'entities'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
data.plot.scatter(x=var, y='solution', ylim=(0,50000));

In [ ]:
var = 'mentions'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
data.plot.scatter(x=var, y='solution', ylim=(0,300000));

In [ ]:
var = 'hashtags'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
data.plot.scatter(x=var, y='solution', ylim=(0,50000));

In [ ]:
var = '#followers'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
data.plot.scatter(x=var, y='solution', ylim=(0,50000));

In [ ]:
var = '#friends'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
data.plot.scatter(x=var, y='solution', ylim=(0,50000));

In [ ]:
len(train_data[(train_data['#friends']<=4000000)&(train_data['#friends']>=2000000)])

In [ ]:
var = '#favorites'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
data.plot.scatter(x=var, y='solution', xlim=(0,10000), ylim=(0,10000));

In [ ]:
#correlation matrix
corrmat = train_data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

In [ ]:
import math
train_data['#friends_new']=train_data['#friends'].map(lambda x: 4486562-x)
train_data['#followers_new']=train_data['#followers'].map(lambda x: 1166612-x)

In [ ]:
#saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'solution')['solution'].index
cm = np.corrcoef(train_data[cols].values.T)
f, ax = plt.subplots(figsize=(12, 9))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
#histogram and normal probability plot
f, ax = plt.subplots(figsize=(12, 4))
sns.distplot(train_data['solution'], fit=norm);
f, ax = plt.subplots(figsize=(12, 4))
res = stats.probplot(train_data['solution'], plot=plt)

In [ ]:
#applying log transformation
train_data['solution'] = np.log(1 + train_data['solution'])

In [ ]:
#histogram and normal probability plot
f, ax = plt.subplots(figsize=(12, 4))
sns.distplot(train_data['solution'], fit=norm);
f, ax = plt.subplots(figsize=(12, 4))
res = stats.probplot(train_data['solution'], plot=plt)

In [ ]:
#histogram and normal probability plot
f, ax = plt.subplots(figsize=(12, 4))
sns.distplot(train_data['#favorites'], fit=norm);
f, ax = plt.subplots(figsize=(12, 4))
res = stats.probplot(train_data['#favorites'], plot=plt)

In [ ]:
#data transformation
train_data['#favorites'] = np.log(1 + train_data['#favorites'])

In [ ]:
#histogram and normal probability plot
f, ax = plt.subplots(figsize=(12, 4))
sns.distplot(train_data['#favorites'], fit=norm);
f, ax = plt.subplots(figsize=(12, 4))
res = stats.probplot(train_data['#favorites'], plot=plt)

In [ ]:
#histogram and normal probability plot
f, ax = plt.subplots(figsize=(12, 4))
sns.distplot(train_data['#friends'], fit=norm);
f, ax = plt.subplots(figsize=(12, 4))
res = stats.probplot(train_data['#friends'], plot=plt)

In [ ]:
train_data['#friends'] = np.log(1 + train_data['#friends'])

In [ ]:
#histogram and normal probability plot
f, ax = plt.subplots(figsize=(12, 4))
sns.distplot(train_data['#friends'], fit=norm);
f, ax = plt.subplots(figsize=(12, 4))
res = stats.probplot(train_data['#friends'], plot=plt)

In [ ]:
#histogram and normal probability plot
f, ax = plt.subplots(figsize=(12, 4))
sns.distplot(train_data['#followers'], fit=norm);
f, ax = plt.subplots(figsize=(12, 4))
res = stats.probplot(train_data['#followers'], plot=plt)

In [ ]:
train_data['#followers'] = np.log(1 + train_data['#followers'])

In [ ]:
#histogram and normal probability plot
f, ax = plt.subplots(figsize=(12, 4))
sns.distplot(train_data['#followers'], fit=norm);
f, ax = plt.subplots(figsize=(12, 4))
res = stats.probplot(train_data['#followers'], plot=plt)

In [ ]:
#saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'solution')['solution'].index
cm = np.corrcoef(train_data[cols].values.T)
f, ax = plt.subplots(figsize=(6, 6))
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
var = '#followers'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
data.plot.scatter(x=var, y='solution');

In [ ]:
var = '#friends'
data = pd.concat([train_data['solution'], train_data[var]], axis=1)
data.plot.scatter(x=var, y='solution');

In [ ]:
sns.residplot(train_data['#friends'], train_data['solution'], lowess=True, color="g")

In [ ]:
train_data.columns

In [ ]:
used_feature = ['timestamp', '#followers', '#friends',
       '#favorites', 'weekday', 'month', 'day', 'hour', 'year', 'sentiment_left',
       'sentiment_right', 'sentiment_sum', 'sentiment_sumabs',
       'sentiment_diff', 'sentiment_div', 'entities_mean', 'entities_max',
       'entities_min', 'entities_std', 'entities_len', 'mentions_len',
       'mentions_value', 'hashtags_len', 'hashtags_value', 'urls_count', 'solution']

In [19]:
from datetime import datetime

In [20]:
datetime_now = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

In [21]:
datetime_now

'2020-07-25_15-07-34'